# Questrade Portfolio Manager

In [1]:
import os
import numpy as np
import pandas as pd
import datetime as dt
from qtrade import Questrade
import matplotlib.pyplot as plt

## Construct Monthly DateList

In [2]:
startDate = '2018-04-01'
endDate = dt.date.today().strftime("%Y-%m-%d")
dtrange = pd.date_range(startDate, endDate, freq='d')
months = pd.Series(dtrange.month)
starts, ends = months.ne(months.shift(1)), months.ne(months.shift(-1))
startEndDates = pd.DataFrame({
    'month_starting_date':
    dtrange[starts].strftime('%Y-%m-%d'),
    'month_ending_date':
    dtrange[ends].strftime('%Y-%m-%d')
})
dateList = startEndDates.values.tolist()

## Authentication to Questrade API

In [10]:
try:
    qtrade = Questrade(token_yaml='access_token.yml')
except:
    qtrade = Questrade(access_code='KUCGAbB-JbI8RsaRS_rXFqwabVXYOkrt0')

In [17]:
pw = open("access_code.txt", "r").read()

'KUCGAbB-JbI8RsaRS_rXFqwabVXYOkrt0'

In [11]:

#qtrade = Questrade(access_code='KUCGAbB-JbI8RsaRS_rXFqwabVXYOkrt0')
# try:
    
# except:
#     if os.path.exists('./access_token.yml'):
#         try:
            
#         except:
#             print("something wrong with refreshing the token")
#     print("please get the new token from Questrade website")
try:
    acctId = qtrade.get_account_id()
except:
    qtrade.refresh_access_token(from_yaml=True)

In [13]:
qtrade.get_account_id()

['51802566']

In [ ]:
output = {}
total_div = 0

for date in dateList:
    start = date[0]
    end = date[1]
    activities = qtrade.get_account_activities(acctId[0], start, end)
    monthly_div = 0
    for activity in activities:
        if activity['type'] == 'Dividends':
            monthly_div = monthly_div + activity['netAmount']
    output[dt.datetime.strptime(start,
                                "%Y-%m-%d").strftime("%Y-%m")] = monthly_div
    total_div = total_div + monthly_div
    #print(f'from {start} to {end}, {monthly_div} was collected.')

# output

In [ ]:
final_df = pd.DataFrame.from_dict(output,
                                  orient='index',
                                  columns=['Monthly_Dividend_Income'])
final_df

## Dividend Income History with Bar Chart Visualization

In [ ]:
final_df.plot(kind='bar',
              title='Monthly Dividend Payments',
              xlabel='Months',
              ylabel='Dividend ($)',
              figsize=(15, 10),
              legend=False,
              ylim=(0, 10))

## Total Dividend Income

In [ ]:
total_div